# Trigger Tower performance plots


   * [Electrons flat pT 2 - 100 GeV](#Electrons-flat-pt-2---100-GeV)
   * [Hadron Gun Flat Pt XX - YY GeV](#Hadron-Gun-Flat-Pt-XX---YY-GeV)



In [112]:
import sys
sys.path.insert(0, '..')
import ROOT
import root_numpy as rnp
import python.l1THistos as histos
import math
import uuid
import pandas as pd

ROOT.enableJSVis()
#ROOT.enableJSVis()

#from drawingTools import *

normalized_histos = list()

In [113]:
# %load ../drawingTools
import ROOT
import math
import uuid

# some useful globals, mainly to deal with ROOT idiosyncrasies
c_idx = 0
p_idx = 0
colors = range(1, 6)
stuff = []
f_idx = 0

ROOT.gStyle.SetPadBottomMargin(0.13)
ROOT.gStyle.SetPadLeftMargin(0.13)
ROOT.gStyle.SetPadRightMargin(0.30)

ROOT.gStyle.SetCanvasBorderMode(0)
ROOT.gStyle.SetCanvasColor(0)
ROOT.gStyle.SetCanvasDefH(600)
ROOT.gStyle.SetCanvasDefW(800)

# define some utility functions
def newCanvas(name=None, title=None, xdiv=0, ydiv=0, form=4):
    global c_idx
    if name is None:
        name = 'c_{}'.format(uuid.uuid4().hex[:6])
        c_idx += 1
    if title is None:
        title = name
    # print name, title
    canvas = ROOT.TCanvas(name, title)
    if(xdiv*ydiv != 0):
        canvas.Divide(xdiv, ydiv)
    global stuff
    stuff.append(canvas)
    return canvas


def draw(plot, options='', text=None):
    c = newCanvas()
    c.cd()
    plot.Draw(options)
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw('same')

    c.Draw()

    return


def drawAll(histograms,
            labels=None,
            options='',
            text=None,
            norm=False,
            logy=False,
            min_y=None,
            max_y=None,
            y_axis_label=None,
            do_ratio=False,
            do_profile=False):
    if len(histograms) == 0:
        print 'ERROR: no histogram in input'
        return -1
    if do_ratio and len(histograms) != 2:
        print 'ERROR: do_ratio option only available when 2 histograms need to be drawn!'
        return -2
    if do_profile and not ('TH2' in histograms[0].ClassName() or 'TH3' in histograms[0].ClassName()):
        print 'ERROR: do_profile option only available with TH2 and TH3 histograms'
        return -3

    # we build the canvas
    return


def drawAll(histograms,
            labels=None,
            options='',
            text=None,
            norm=False,
            logy=False,
            min_y=None,
            max_y=None,
            y_axis_label=None,
            do_ratio=False,
            do_profile=False):
    if len(histograms) == 0:
        print 'ERROR: no histogram in input'
        return -1
    if do_ratio and len(histograms) != 2:
        print 'ERROR: do_ratio option only available when 2 histograms need to be drawn!'
        return -2
    if do_profile and not ('TH2' in histograms[0].ClassName() or 'TH3' in histograms[0].ClassName()):
        print 'ERROR: do_profile option only available with TH2 and TH3 histograms'
        return -3

    # we build the canvas
    return 0


def getLegend(x1=0.7, y1=0.71, x2=0.95, y2=0.85):
    global stuff
    legend = ROOT.TLegend(x1, y1, x2, y2)
    stuff.append(legend)
    legend.SetFillColor(0)
    legend.SetFillStyle(0)
    legend.SetBorderSize(0)
    legend.SetTextSize(0.05)
    return legend


def drawAndProfileX(plot2d, miny=None, maxy=None, do_profile=True, options='', text=None):
    global p_idx
    if miny and maxy:
        plot2d.GetYaxis().SetRangeUser(miny, maxy)
    c = newCanvas()
    c.SetGrid(1, 1)
    c.cd()
    plot2d.Draw(options)
    ROOT.gPad.SetGrid(1, 1)
    ROOT.gStyle.SetGridColor(15)

    if do_profile:
        profname = plot2d.GetName()+'_prof_'+str(p_idx)
        p_idx += 1
        firstbin = 1
        lastbin = -1
        prof = plot2d.ProfileX(profname, firstbin, lastbin, 's')
        prof.SetMarkerColor(2)
        prof.SetLineColor(2)
        prof.Draw('same')

    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw('same')

    c.Draw()


def getText(text, ndc_x, ndc_y):
    global stuff
    rtext = ROOT.TLatex(ndc_x, ndc_y, text)
    stuff.append(rtext)
    rtext.SetNDC(True)
    # rtext.SetTextFont(40)
    rtext.SetTextSize(0.03)
    return rtext



def drawSame(histograms,
             labels,
             options='',
             norm=False,
             logy=False,
             min_y=None,
             max_y=None,
             text=None,
             y_axis_label=None,
             x_axis_label=None,
             v_lines=None,
             h_lines=None):
    global colors
    global stuff
    c = newCanvas(title=histograms[0].GetName())
    c.cd()
    leg = getLegend()

    max_value = max_y
    min_value = min_y
    
    newHistos = []
    for hidx, hist in enumerate(histograms):
        hist.SetLineColor(colors[hidx])
        hist.SetStats(False)
        if norm:
            nhist = hist.DrawNormalized('same'+','+options, 1.)
            newHistos.append(nhist)
        else:
            if hidx:
                hist.Draw('same'+','+options)
            else:
                hist.Draw(options)
            newHistos.append(hist)
        leg.AddEntry(histograms[hidx], labels[hidx], 'l')

    if min_y is None:
        min_value = min([hist.GetBinContent(hist.GetMinimumBin()) for hist in newHistos])
    if max_y is None:
        max_value = max([hist.GetBinContent(hist.GetMaximumBin()) for hist in newHistos])*1.2
#     print min_value, max_value

    newHistos[0].GetYaxis().SetRangeUser(min_value, max_value)
    if y_axis_label:
        newHistos[0].GetYaxis().SetTitle(y_axis_label)
    if x_axis_label:
            newHistos[0].GetXaxis().SetTitle(x_axis_label)

    leg.Draw()
    c.Draw()
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw("same")
    if logy:
        c.SetLogy()

    if v_lines:
        for v_line_x in v_lines:
            aline = ROOT.TLine(v_line_x, c.GetUymin(), v_line_x,  c.GetUymax())
            aline.SetLineStyle(2)
            aline.Draw("same")
            stuff.append(aline)
    if h_lines:
        for h_line_y in h_lines:
            aline = ROOT.TLine(c.GetUxmin(), h_line_y, c.GetUxmax(),  h_line_y)
            aline.SetLineStyle(2)
            aline.Draw("same")
            stuff.append(aline)
    c.Update()


def drawProfileX(histograms, labels, options=''):
    profiles = [hist.ProfileX() for hist in histograms]
    drawSame(profiles, labels, options)


def drawSeveral(histograms, labels, options='', do_profile=False, miny=None, maxy=None, text=None):
    ydiv = int(math.ceil(float(len(histograms))/2))
    for hidx in range(0, len(histograms)):
        newtext = labels[hidx]
        if text:
            newtext = '{}: {}'.format(labels[hidx], text)
        if do_profile:
            drawAndProfileX(histograms[hidx], miny=miny, maxy=maxy, options=options, do_profile=do_profile, text=newtext)
        else:
            draw(histograms[hidx], options=options, text=newtext)


def drawProfileRatio(prof1, prof2, ymin=None, ymax=None, text=None):
    hist1 = prof1.ProjectionX(uuid.uuid4().hex[:6])
    hist2 = prof2.ProjectionX(uuid.uuid4().hex[:6])
    hist1.Divide(hist2)
    draw(hist1)
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw("same")

    if ymin is not None and ymax is not None:
        hist1.GetYaxis().SetRangeUser(ymin, ymax)
    ROOT.gPad.Update()


# mean+-nsigmas*RMS.
def drawGaussFit(histo, nsigmas, min, max):
    minfit = histo.GetMean() - nsigmas*histo.GetRMS()
    maxfit = histo.GetMean() + nsigmas*histo.GetRMS()
    drawGFit(histo, min, max, minfit, maxfit)


# Fit a histogram in the range (minfit, maxfit) with a gaussian and
# draw it in the range (min, max)
def drawGFit(histo, min, max, minfit, maxfit):
    # static int i = 0
    # i++
    # gPad->SetGrid(1,1);
    # gStyle->SetGridColor(15);
    histo.GetXaxis().SetRangeUser(min, max)
    global f_idx
    nameF1 = "g{}".format(f_idx)
    f_idx +=1
    g1 = ROOT.TF1(nameF1, "gaus", minfit, maxfit)
    g1.SetLineColor(2)
    g1.SetLineWidth(2)
    histo.Fit(g1,"R")


def drawGraphsSame(histograms,
                   labels,
                   options='',
                   norm=False,
                   logy=False,
                   min_y=None,
                   max_y=None,
                   text=None):
    global colors
    c = newCanvas()
    c.cd()
    leg = getLegend()

    for hidx in range(0, len(histograms)):
        histograms[hidx].SetLineColor(colors[hidx])
        histograms[hidx].Draw('same'+','+options)
        leg.AddEntry(histograms[hidx], labels[hidx], 'l')

    max_value = max_y
    min_value = min_y
    if min_y is None:
        min_value = min([hist.GetBinContent(hist.GetMinimumBin()) for hist in histograms])
    if max_y is None:
        max_value = max([hist.GetBinContent(hist.GetMaximumBin()) for hist in histograms])*1.2
    histograms[0].GetYaxis().SetRangeUser(min_value, max_value)
    leg.Draw()
    c.Draw()
    if logy:
        c.SetLogy()
    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw("same")
    c.Update()


In [114]:
# %load samples.py
import ROOT
import pandas as pd
import python.selections as selections

version = 'v57a'

files = {}
file_keys = {}


class RootFile:
    def __init__(self, file_name):
        global file
        self.file_name = file_name
        if self.file_name not in files.keys():
            print 'get file: {}'.format(self.file_name)
            files[self.file_name] = ROOT.TFile(self.file_name)
        self._file = files[self.file_name]
        self._file_keys = None

    def cd(self):
        self._file.cd()

    def GetListOfKeys(self):
        global file_keys
        if self.file_name not in file_keys.keys():
            print 'get list'
            file_keys[self.file_name] = self._file.GetListOfKeys()
        self._file_keys = file_keys[self.file_name]
        return self._file_keys


class Sample():
    def __init__(self, name, label, version=None, type=None):
        self.name = name
        self.label = label
        if version:
            version = '_'+version
        else:
            version = ''
        self.histo_filename = '../plots1/histos_{}{}.root'.format(self.name, version)
        self.histo_file = ROOT.TFile(self.histo_filename, 'r')
        self.type = type


# sample_names = ['ele_flat2to100_PU0',
#                 'ele_flat2to100_PU200',
#                 'photonPt35_PU0',
#                 'photonPt35_PU200']


def get_label_dict(selections):
    dictionary = {}
    for sel in selections:
        dictionary[sel.name] = sel.label
    return dictionary


class HProxy:
    def __init__(self, classtype, tp, tp_sel, gen_sel, root_file):
        self.classtype = classtype
        self.tp = tp
        self.tp_sel = tp_sel
        self.gen_sel = gen_sel
        self.root_file = root_file
        self.instance = None

    def get(self):
        if self.instance is None:
            name = '{}_{}_{}'.format(self.tp, self.tp_sel, self.gen_sel)
            if self.gen_sel is None:
                name = '{}_{}'.format(self.tp, self.tp_sel)
            self.instance = self.classtype(name, self.root_file)
        return self.instance


class HPlot:
    def __init__(self, samples, tp_sets, tp_selections, gen_selections):
        self.tp_sets = tp_sets
        self.tp_selections = tp_selections
        self.gen_selections = gen_selections
        self.pus = []
        self.labels_dict = {}

        for sample in samples:
            self.pus.append(sample.label)
            self.labels_dict[sample.type] = sample.type

        self.data = pd.DataFrame(columns=['sample', 'pu', 'tp', 'tp_sel', 'gen_sel', 'classtype', 'histo'])

        self.labels_dict.update(tp_sets)
        self.labels_dict.update(tp_selections)
        self.labels_dict.update(gen_selections)
        self.labels_dict.update({'PU0': 'PU0', 'PU200': 'PU200'})

    def cache_histo(self,
                    classtype,
                    samples,
                    pus,
                    tps,
                    tp_sels,
                    gen_sels):
        if gen_sels is None:
            gen_sels = [None]

        for sample in samples:
            print sample
            for tp in tps:
                for tp_sel in tp_sels:
                    for gen_sel in gen_sels:
                        print sample, tp, tp_sel, gen_sel
                        self.data = self.data.append({'sample': sample.type,
                                                      'pu': sample.label,
                                                      'tp': tp,
                                                      'tp_sel': tp_sel,
                                                      'gen_sel': gen_sel,
                                                      'classtype': classtype,
                                                      'histo': HProxy(classtype, tp, tp_sel, gen_sel, sample.histo_file)},
                                                     ignore_index=True)

    def get_histo(self,
                  classtype,
                  sample=None,
                  pu=None,
                  tp=None,
                  tp_sel=None,
                  gen_sel=None):
        histo = None
        labels = []
        text = ''

        query = '(pu == @pu) & (tp == @tp) & (tp_sel == @tp_sel) & (classtype == @classtype)'
        if gen_sel is not None:
            query += ' & (gen_sel == @gen_sel)'
        else:
            query += ' & (gen_sel.isnull())'
        if sample is not None:
            query += '& (sample == @sample)'
        histo_df = self.data.query(query)

        if histo_df.empty:
            print 'No match found for: pu: {}, tp: {}, tp_sel: {}, gen_sel: {}, classtype: {}'.format(pu, tp, tp_sel, gen_sel, classtype)
            return None, None, None
#         print histo_df

        field_counts = histo_df.apply(lambda x: len(x.unique()))
        label_fields = []
        text_fields = []
        # print field_counts
        for field in field_counts.iteritems():
            if(field[1] > 1 and field[0] != 'histo'):
                label_fields.append(field[0])
            if(field[1] == 1 and field[0] != 'histo' and field[0] != 'classtype' and field[0] != 'sample'):
                if(gen_sel is None and field[0] == 'gen_sel'):
                    continue
                text_fields.append(field[0])
#         print 'label fields: {}'.format(label_fields)
#         print 'text fields: {}'.format(text_fields)

        for item in histo_df[label_fields].iterrows():
            labels.append(', '.join([self.labels_dict[tx] for tx in item[1].values if self.labels_dict[tx] != '']))

        # print labels
        text = ', '.join([self.labels_dict[fl] for fl in histo_df[text_fields].iloc[0].values if self.labels_dict[fl] != ''])
        histo = [his.get() for his in histo_df['histo'].values]
        return histo, labels, text


# -------------------------------------------------------------------------

samples_ele = [
    Sample('ele_flat2to100_PU0', 'PU0', version, 'ele'),
    Sample('ele_flat2to100_PU200', 'PU200', 'V57', 'ele')
    ]

samples_photons = [
    Sample('photon_flat8to150_PU0', 'PU0', version, 'photon'),
    Sample('photon_flat8to150_PU200', 'PU200', version, 'photon')
    ]

samples_pions = [
    Sample('pion_flat2to100_PU0', 'PU0', version, 'pions'),
    ]

samples_nugus = [
    Sample('nugun_alleta_pu0', 'PU0', version, 'mb'),
    Sample('nugun_alleta_pu200', 'PU200', version, 'mb')
    ]

samples_nugunrates = [
    Sample('nugun_alleta_pu200', 'PU200', version, 'mb')
    ]


tpsets = {'DEF': 'NNDR',
          'DEFCalib': 'NNDR Calib v1'}

tpset_selections = {}
gen_selections = {}
samples = []

# tpset_selections.update(get_label_dict(tp_rate_selections))
tpset_selections.update(get_label_dict(selections.tp_match_selections))
gen_selections.update(get_label_dict(selections.gen_part_selections))
gen_selections.update({'nomatch': ''})

Error in <TFile::TFile>: file ../plots1/histos_photon_flat8to150_PU0_v57a.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_photon_flat8to150_PU200_v57a.root does not exist


In [115]:
tpsets = {'TT': 'TC TT',
          'TTCl': 'TC TT Cl.',
          'SimTT': 'Sim TT',
          'SimTTCl': 'Sim TT Cl.',
          'HgcrocTT': 'HGCROC TT',
          'HgcrocTTCl': 'HGCROC TT Cl.',
          'WaferTT': 'Wafer TT',
          'WaferTTCl': 'Wafer TT Cl.'
         }
tpset_selections = {'all': ''}
samples = samples_ele
samples += samples_nugus

In [116]:
tpsets

{'HgcrocTT': 'HGCROC TT',
 'HgcrocTTCl': 'HGCROC TT Cl.',
 'SimTT': 'Sim TT',
 'SimTTCl': 'Sim TT Cl.',
 'TT': 'TC TT',
 'TTCl': 'TC TT Cl.',
 'WaferTT': 'Wafer TT',
 'WaferTTCl': 'Wafer TT Cl.'}

In [117]:
tpset_selections

{'all': ''}

In [118]:
gen_selections

{'GEN': '',
 'GENEtaB': '1.52 < |#eta^{GEN}| <= 1.7',
 'GENEtaBC': '1.52 < |#eta^{GEN}| <= 2.4',
 'GENEtaBCD': '1.52 < |#eta^{GEN}| <= 2.8',
 'GENEtaC': '1.7 < |#eta^{GEN}| <= 2.4',
 'GENEtaD': '2.4 < |#eta^{GEN}| <= 2.8',
 'GENPt10': 'p_{T}^{GEN}>=10GeV',
 'GENPt20': 'p_{T}^{GEN}>=20GeV',
 'GENPt30': 'p_{T}^{GEN}>=30GeV',
 'GENPt40': 'p_{T}^{GEN}>=40GeV',
 'nomatch': ''}

In [119]:
samples

[<__main__.Sample instance at 0x118998518>,
 <__main__.Sample instance at 0x118b0b1b8>,
 <__main__.Sample instance at 0x118998368>,
 <__main__.Sample instance at 0x118b0b248>]

In [120]:
%%time

hplot = HPlot(samples, tpsets, tpset_selections, gen_selections)


# hplot.cache_histo(classtype=histos.TriggerTowerHistos, 
#                   samples=samples,
#                   pus=[],
#                   tps=tpsets,
#                   tp_sels=tpset_selections,
#                   gen_sels=['nomatch'])


hplot.cache_histo(classtype=histos.TriggerTowerHistos, 
                  samples=samples,
                  pus=[],
                  tps=tpsets,
                  tp_sels=tpset_selections,
                  gen_sels=gen_selections)


# hplot.cache_histo(classtype=histos.TriggerTowerResoHistos, 
#                   samples=samples,
#                   pus=[],
#                   tps=tpsets,
#                   tp_sels=tpset_selections,
#                   gen_sels=gen_selections)




<__main__.Sample instance at 0x118998518>
<__main__.Sample instance at 0x118998518> WaferTT all GENPt40
<__main__.Sample instance at 0x118998518> WaferTT all GENEtaC
<__main__.Sample instance at 0x118998518> WaferTT all GENEtaB
<__main__.Sample instance at 0x118998518> WaferTT all GENEtaD
<__main__.Sample instance at 0x118998518> WaferTT all GENEtaBC
<__main__.Sample instance at 0x118998518> WaferTT all GENPt30
<__main__.Sample instance at 0x118998518> WaferTT all GENPt20
<__main__.Sample instance at 0x118998518> WaferTT all GENEtaBCD
<__main__.Sample instance at 0x118998518> WaferTT all GENPt10
<__main__.Sample instance at 0x118998518> WaferTT all nomatch
<__main__.Sample instance at 0x118998518> WaferTT all GEN
<__main__.Sample instance at 0x118998518> HgcrocTTCl all GENPt40
<__main__.Sample instance at 0x118998518> HgcrocTTCl all GENEtaC
<__main__.Sample instance at 0x118998518> HgcrocTTCl all GENEtaB
<__main__.Sample instance at 0x118998518> HgcrocTTCl all GENEtaD
<__main__.Sample 

<__main__.Sample instance at 0x118b0b1b8> TT all GENEtaBCD
<__main__.Sample instance at 0x118b0b1b8> TT all GENPt10
<__main__.Sample instance at 0x118b0b1b8> TT all nomatch
<__main__.Sample instance at 0x118b0b1b8> TT all GEN
<__main__.Sample instance at 0x118b0b1b8> WaferTTCl all GENPt40
<__main__.Sample instance at 0x118b0b1b8> WaferTTCl all GENEtaC
<__main__.Sample instance at 0x118b0b1b8> WaferTTCl all GENEtaB
<__main__.Sample instance at 0x118b0b1b8> WaferTTCl all GENEtaD
<__main__.Sample instance at 0x118b0b1b8> WaferTTCl all GENEtaBC
<__main__.Sample instance at 0x118b0b1b8> WaferTTCl all GENPt30
<__main__.Sample instance at 0x118b0b1b8> WaferTTCl all GENPt20
<__main__.Sample instance at 0x118b0b1b8> WaferTTCl all GENEtaBCD
<__main__.Sample instance at 0x118b0b1b8> WaferTTCl all GENPt10
<__main__.Sample instance at 0x118b0b1b8> WaferTTCl all nomatch
<__main__.Sample instance at 0x118b0b1b8> WaferTTCl all GEN
<__main__.Sample instance at 0x118b0b1b8> HgcrocTT all GENPt40
<__main_

<__main__.Sample instance at 0x118b0b248> SimTT all GENPt30
<__main__.Sample instance at 0x118b0b248> SimTT all GENPt20
<__main__.Sample instance at 0x118b0b248> SimTT all GENEtaBCD
<__main__.Sample instance at 0x118b0b248> SimTT all GENPt10
<__main__.Sample instance at 0x118b0b248> SimTT all nomatch
<__main__.Sample instance at 0x118b0b248> SimTT all GEN
<__main__.Sample instance at 0x118b0b248> TTCl all GENPt40
<__main__.Sample instance at 0x118b0b248> TTCl all GENEtaC
<__main__.Sample instance at 0x118b0b248> TTCl all GENEtaB
<__main__.Sample instance at 0x118b0b248> TTCl all GENEtaD
<__main__.Sample instance at 0x118b0b248> TTCl all GENEtaBC
<__main__.Sample instance at 0x118b0b248> TTCl all GENPt30
<__main__.Sample instance at 0x118b0b248> TTCl all GENPt20
<__main__.Sample instance at 0x118b0b248> TTCl all GENEtaBCD
<__main__.Sample instance at 0x118b0b248> TTCl all GENPt10
<__main__.Sample instance at 0x118b0b248> TTCl all nomatch
<__main__.Sample instance at 0x118b0b248> TTCl al

In [68]:
hplot.data[:10]

,sample,pu,tp,tp_sel,gen_sel,classtype,histo
0,ele,PU0,WaferTT,all,GENPt40,python.l1THistos.TriggerTowerHistos,<__main__.HProxy instance at 0x118255200>
1,ele,PU0,WaferTT,all,GENEtaC,python.l1THistos.TriggerTowerHistos,<__main__.HProxy instance at 0x118234368>
2,ele,PU0,WaferTT,all,GENEtaB,python.l1THistos.TriggerTowerHistos,<__main__.HProxy instance at 0x118234cf8>
3,ele,PU0,WaferTT,all,GENEtaD,python.l1THistos.TriggerTowerHistos,<__main__.HProxy instance at 0x117764fc8>
4,ele,PU0,WaferTT,all,GENEtaBC,python.l1THistos.TriggerTowerHistos,<__main__.HProxy instance at 0x118234518>
5,ele,PU0,WaferTT,all,GENPt30,python.l1THistos.TriggerTowerHistos,<__main__.HProxy instance at 0x11844f518>
6,ele,PU0,WaferTT,all,GENPt20,python.l1THistos.TriggerTowerHistos,<__main__.HProxy instance at 0x118255b90>
7,ele,PU0,WaferTT,all,GENEtaBCD,python.l1THistos.TriggerTowerHistos,<__main__.HProxy instance at 0x118234b90>
8,ele,PU0,WaferTT,all,GENPt10,python.l1THistos.TriggerTowerHistos,<__main__.HProxy instance at 0x118262ea8>
9,ele,PU0,WaferTT,all,nomatch,python.l1THistos.TriggerTowerHistos,<__main__.HProxy instance at 0x118260fc8>


In [69]:
hplot.data.query('gen_sel == "nomatch"')

,sample,pu,tp,tp_sel,gen_sel,classtype,histo
9,ele,PU0,WaferTT,all,nomatch,python.l1THistos.TriggerTowerHistos,<__main__.HProxy instance at 0x118260fc8>
20,ele,PU0,HgcrocTTCl,all,nomatch,python.l1THistos.TriggerTowerHistos,<__main__.HProxy instance at 0x1187e95f0>
31,ele,PU0,SimTTCl,all,nomatch,python.l1THistos.TriggerTowerHistos,<__main__.HProxy instance at 0x118452128>
42,ele,PU0,SimTT,all,nomatch,python.l1THistos.TriggerTowerHistos,<__main__.HProxy instance at 0x118823998>
53,ele,PU0,TTCl,all,nomatch,python.l1THistos.TriggerTowerHistos,<__main__.HProxy instance at 0x11823db00>
64,ele,PU0,TT,all,nomatch,python.l1THistos.TriggerTowerHistos,<__main__.HProxy instance at 0x118452cf8>
75,ele,PU0,WaferTTCl,all,nomatch,python.l1THistos.TriggerTowerHistos,<__main__.HProxy instance at 0x1189a1098>
86,ele,PU0,HgcrocTT,all,nomatch,python.l1THistos.TriggerTowerHistos,<__main__.HProxy instance at 0x1188235f0>
97,ele,PU200,WaferTT,all,nomatch,python.l1THistos.TriggerTowerHistos,<__main__.HProxy instance at 0x11841e320>
108,ele,PU200,HgcrocTTCl,all,nomatch,python.l1THistos.TriggerTowerHistos,<__main__.HProxy instance at 0x118258e18>


In [70]:
tps = ['TT', 'SimTT', 'HgcrocTT', 'WaferTT']


## Electrons flat pt 2 - 100 GeV

In [71]:
samples = samples_ele
labels = [sample.label for sample in samples]
tps = ['TT', 'HgcrocTT', 'WaferTT', 'SimTT']

tp_select = ['all']
# # Extract # of events for normalization
# nevents_ee = [histo.GetBinContent(3) for histo in hGen_reachedEE]
# print nevents_ee
sample_type = 'ele'

### All Towers (no match)

In [73]:

for tp in tps:
    for tp_sel in tp_select:
        hsets, labels, text = hplot.get_histo(histos.TriggerTowerHistos, sample_type, ['PU0', 'PU200'], tp, tp_sel, ['nomatch'])            
#         print hsets
        drawSame([his.h_pt for his in hsets], labels, norm=True, text=text, logy=True)


In [74]:

for pu in ['PU0', 'PU200']:
    for tp_sel in tp_select:
        hsets, labels, text = hplot.get_histo(histos.TriggerTowerHistos, sample_type, pu, ['TT', 'HgcrocTT', 'WaferTT'], tp_sel, ['nomatch'])            
        drawSame([his.h_pt for his in hsets], labels, norm=True, text=text, logy=True)


#### Et Had and EM

In [75]:

for tp in tps:
    for tp_sel in tp_select:
        hsets, labels, text = hplot.get_histo(histos.TriggerTowerHistos, sample_type, ['PU0', 'PU200'], tp, tp_sel, ['nomatch'])            
        drawSame([his.h_etEm for his in hsets], labels, norm=True, text=text, logy=True)


In [76]:

for tp in tps:
    for tp_sel in tp_select:
        hsets, labels, text = hplot.get_histo(histos.TriggerTowerHistos, sample_type, ['PU0', 'PU200'], tp, tp_sel, ['nomatch'])            
        drawSame([his.h_etHad for his in hsets], labels, norm=True, text=text, logy=True)


Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale


In [77]:

for pu in ['PU0', 'PU200']:
    for tp_sel in tp_select:
        hsets, labels, text = hplot.get_histo(histos.TriggerTowerHistos, sample_type, pu, ['TT', 'HgcrocTT', 'WaferTT'], tp_sel, ['nomatch'])            
        drawSame([his.h_etEm for his in hsets], labels, norm=True, text=text, logy=True)


In [78]:

for pu in ['PU0', 'PU200']:
    for tp_sel in tp_select:
        hsets, labels, text = hplot.get_histo(histos.TriggerTowerHistos, sample_type, pu, ['TT', 'HgcrocTT', 'WaferTT'], tp_sel, ['nomatch'])            
        drawSame([his.h_etHad for his in hsets], labels, norm=True, text=text, logy=True)


Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale


#### Eta

In [79]:

for tp in tps:
    for tp_sel in tp_select:
        hsets, labels, text = hplot.get_histo(histos.TriggerTowerHistos, sample_type, ['PU0', 'PU200'], tp, tp_sel, ['nomatch'])  
        drawSame([his.h_eta for his in hsets], labels, norm=True, text=text, logy=True)


Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale


In [80]:

for tp in tps:
    for tp_sel in tp_select:
        hsets, labels, text = hplot.get_histo(histos.TriggerTowerHistos, sample_type, ['PU0', 'PU200'], tp, tp_sel, ['nomatch'])            
        drawSeveral([his.h_etVieta for his in hsets], labels, text=text, options='COLZ', do_profile=True)


In [81]:

for pu in ['PU0', 'PU200']:
    for tp_sel in tp_select:
        hsets, labels, text = hplot.get_histo(histos.TriggerTowerHistos, sample_type, pu, ['TT', 'HgcrocTT', 'WaferTT'], tp_sel, ['nomatch'])            
        drawSame([his.h_sumEt for his in hsets], labels, norm=True, text=text, logy=True)


Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale


In [85]:

for pu in ['PU0', 'PU200']:
    for tp_sel in tp_select:
        hsets, labels, text = hplot.get_histo(histos.TriggerTowerHistos, sample_type, pu, ['TT', 'HgcrocTT', 'WaferTT'], tp_sel, ['nomatch'])            
        drawSame([his.h_sumEtCentral for his in hsets], labels, norm=True, text=text, logy=True)


Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale


### Towers matched to GEN particles

In [86]:



for tp in tps:
    for tp_sel in tp_select:
        for gen_sel in ['GEN', 'GENEtaBCD']:
            hsets, labels, text = hplot.get_histo(histos.TriggerTowerHistos, sample_type, ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            dir(hsets[0])
            drawSame([his.h_pt for his in hsets], labels, norm=True, text=text)


In [87]:

for tp in tps:
    for tp_sel in tp_select:
        for gen_sel in ['GEN']:
            hsets, labels, text = hplot.get_histo(histos.TriggerTowerHistos, sample_type, ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            dir(hsets[0])
            drawSame([his.h_eta for his in hsets], labels, norm=True, text=text)


## $\nu$ guns

In [121]:
samples = samples_ele
labels = [sample.label for sample in samples]
tps = ['TT', 'HgcrocTT', 'WaferTT', 'SimTT']

tp_select = ['all']
# # Extract # of events for normalization
# nevents_ee = [histo.GetBinContent(3) for histo in hGen_reachedEE]
# print nevents_ee
sample_type = 'mb'

### All Towers (no match)

In [128]:

for tp in tps:
    for tp_sel in tp_select:
        hsets, labels, text = hplot.get_histo(histos.TriggerTowerHistos, sample_type, ['PU0', 'PU200'], tp, tp_sel, ['nomatch'])            
        drawSame([his.h_pt for his in hsets], labels, norm=True, text=text, logy=True)


111371916.0
256605199.0
108232818.0
256525653.0
73065815.0
242408288.0
0.0
143889436.0


TypeError: none of the 2 overloaded methods succeeded. Full details:
  attempt to access a null-pointer
  int TH1::GetMinimumBin(int& locmix, int& locmiy, int& locmiz) =>
    takes at least 3 arguments (0 given)

Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <TH1F::DrawNormalized>: Sum of weights is null. Cannot normalize histogram: SimTT_all_nomatch_pt


In [ ]:

for pu in ['PU0', 'PU200']:
    for tp_sel in tp_select:
        hsets, labels, text = hplot.get_histo(histos.TriggerTowerHistos, sample_type, pu, ['TT', 'HgcrocTT', 'WaferTT'], tp_sel, ['nomatch'])            
        drawSame([his.h_pt for his in hsets], labels, norm=True, text=text, logy=True)


#### Et Had and EM

In [100]:

for tp in tps:
    for tp_sel in tp_select:
        hsets, labels, text = hplot.get_histo(histos.TriggerTowerHistos, sample_type, ['PU0', 'PU200'], tp, tp_sel, ['nomatch'])            
        drawSame([his.h_etEm for his in hsets], labels, norm=True, text=text, logy=True)


TypeError: none of the 2 overloaded methods succeeded. Full details:
  attempt to access a null-pointer
  int TH1::GetMinimumBin(int& locmix, int& locmiy, int& locmiz) =>
    takes at least 3 arguments (0 given)

Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <TH1F::DrawNormalized>: Sum of weights is null. Cannot normalize histogram: SimTT_all_nomatch_etEm


In [ ]:

for tp in tps:
    for tp_sel in tp_select:
        hsets, labels, text = hplot.get_histo(histos.TriggerTowerHistos, 
                                              sample_type, 
                                              ['PU0', 'PU200'], 
                                              tp, tp_sel, ['nomatch'])            
        drawSame([his.h_etHad for his in hsets], labels, norm=True, text=text, logy=True)


In [129]:

for pu in ['PU0', 'PU200']:
    for tp_sel in tp_select:
        hsets, labels, text = hplot.get_histo(histos.TriggerTowerHistos, 
                                              sample_type, 
                                              pu, ['TT', 'HgcrocTT', 'WaferTT'], tp_sel, ['nomatch'])            
        drawSame([his.h_etEm for his in hsets], labels, norm=True, text=text, logy=True)


Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale


In [130]:

for pu in ['PU0', 'PU200']:
    for tp_sel in tp_select:
        hsets, labels, text = hplot.get_histo(histos.TriggerTowerHistos, sample_type, pu, ['TT', 'HgcrocTT', 'WaferTT'], tp_sel, ['nomatch'])            
        drawSame([his.h_etHad for his in hsets], labels, norm=True, text=text, logy=True)


Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale


#### Eta

In [131]:

for tp in tps:
    for tp_sel in tp_select:
        hsets, labels, text = hplot.get_histo(histos.TriggerTowerHistos, sample_type, ['PU0', 'PU200'], 
                                              tp, tp_sel, ['nomatch'])  
        drawSame([his.h_eta for his in hsets], labels, norm=True, text=text, logy=True)


TypeError: none of the 2 overloaded methods succeeded. Full details:
  attempt to access a null-pointer
  int TH1::GetMinimumBin(int& locmix, int& locmiy, int& locmiz) =>
    takes at least 3 arguments (0 given)

Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <TH1F::DrawNormalized>: Sum of weights is null. Cannot normalize histogram: SimTT_all_nomatch_eta


In [132]:

for tp in tps:
    for tp_sel in tp_select:
        hsets, labels, text = hplot.get_histo(histos.TriggerTowerHistos, sample_type, ['PU0', 'PU200'], tp, tp_sel, ['nomatch'])            
        drawSeveral([his.h_etVieta for his in hsets], labels, text=text, options='COLZ', do_profile=True)


In [135]:
for pu in ['PU200']:
    for tp_sel in tp_select:
        hsets, labels, text = hplot.get_histo(histos.TriggerTowerHistos, sample_type, pu, ['TT', 'HgcrocTT', 'WaferTT', 'SimTT'], tp_sel, ['nomatch'])            
        drawSame([his.h_etVieta.ProjectionY(uuid.uuid4().hex[:6], 2, 2) for his in hsets], labels, text=text+', iEta==3', norm=True)


In [134]:
for pu in ['PU200']:
    for tp_sel in tp_select:
        hsets, labels, text = hplot.get_histo(histos.TriggerTowerHistos, sample_type, pu, ['TT', 'HgcrocTT', 'WaferTT', 'SimTT'], tp_sel, ['nomatch'])            
        drawSame([his.h_etVieta.ProjectionY(uuid.uuid4().hex[:6], 10, 10) for his in hsets], labels, text=text+', iEta==11', norm=True)


In [133]:
for pu in ['PU200']:
    for tp_sel in tp_select:
        hsets, labels, text = hplot.get_histo(histos.TriggerTowerHistos, sample_type, pu, ['TT', 'HgcrocTT', 'WaferTT', 'SimTT'], tp_sel, ['nomatch'])            
        drawSame([his.h_etEmVieta.ProjectionY(uuid.uuid4().hex[:6], 16, 16) for his in hsets], labels, text=text+', iEta==17', norm=True, logy=True)


In [96]:

for tp in tps:
    for tp_sel in tp_select:
        hsets, labels, text = hplot.get_histo(histos.TriggerTowerHistos, sample_type, ['PU0', 'PU200'], tp, tp_sel, ['nomatch'])            
        drawSeveral([his.h_etEmVieta for his in hsets], labels, text=text, options='COLZ', do_profile=True)


In [108]:

for tp in tps:
    for tp_sel in tp_select:
        hsets, labels, text = hplot.get_histo(histos.TriggerTowerHistos, sample_type, ['PU0','PU200'], tp, tp_sel, ['nomatch'])            
        drawSeveral([his.h_etHadVieta for his in hsets], labels, text=text, options='COLZ', do_profile=True)


In [110]:

for pu in ['PU200']:
    for tp_sel in tp_select:
        hsets, labels, text = hplot.get_histo(histos.TriggerTowerHistos, sample_type, pu, ['TT', 'HgcrocTT', 'WaferTT', 'SimTT'], tp_sel, ['nomatch'])            
        drawSame([his.h_sumEt for his in hsets], labels, norm=True, text=text, logy=True)


Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale


In [99]:

for pu in ['PU0', 'PU200']:
    for tp_sel in tp_select:
        hsets, labels, text = hplot.get_histo(histos.TriggerTowerHistos, sample_type, pu, ['TT', 'HgcrocTT', 'WaferTT', 'SimTT'], tp_sel, ['nomatch'])            
        drawSame([his.h_sumEtCentral for his in hsets], labels, norm=True, text=text, logy=True)


TypeError: none of the 2 overloaded methods succeeded. Full details:
  attempt to access a null-pointer
  int TH1::GetMinimumBin(int& locmix, int& locmiy, int& locmiz) =>
    takes at least 3 arguments (0 given)

Error in <TH1F::DrawNormalized>: Sum of weights is null. Cannot normalize histogram: SimTT_all_nomatch_sumEtCentral


## Hadron Gun Flat Pt XX - YY GeV